In Approach one, we will process the data using a ratio rather than the usual normalization. For example we will take all the cashflow values as a ratio to net income, and all the balance sheet values as a ratio to current assets. We can compare the performance between the two.

In [8]:
import os
import pandas as pd
import dateutil.relativedelta
from datetime import datetime


parent_dir = os.path.join(os.getcwd() , "Data/Raw")
OLS_input = {}
#basic_info_check = ["ebitdaMargins" , "profitMargins"  , "grossMargins", "debtToEquity", "returnOnEquity" ]
cashflow_info_check = ["Change To Liabilities", "Total Cash From Operating Activities", "Net Borrowings" , "Total Cashflows From Investing Activities" , "Investments", "Net Income"]
balance_sheet_info_check = ["Total Liab" , "Total Assets", "Retained Earnings", "Cash", "Net Receivables", "Long Term Debt", "Short Long Term Debt", "Total Current Assets"]

for file in os.listdir(parent_dir):
    path = os.path.join(parent_dir , file)
        

    ### Get variables from Cash Flow Statement ###
    cashflow_info = pd.read_csv(path + "/cashflow.csv")
    balance_sheet = pd.read_csv(path + "/stock_balance_sheet.csv")
    
    if(cashflow_info.empty or balance_sheet.empty):
        continue
 

    if(len(cashflow_info.columns) != len(balance_sheet.columns)):
        print("EEROROR")
    

    for info_item in cashflow_info_check:
        
        if(info_item in cashflow_info["Unnamed: 0"].values):

            data =  cashflow_info[cashflow_info["Unnamed: 0"] == info_item].values[0][1::]
            data = data.tolist()
            if len(data) != 4:

                print("Cash flow OOF" , file)
            # data = transformer.transform(data.reshape(1, -1))
            # data = data.tolist()

        else:
            data = [None] * (len(balance_sheet.columns)-1)

        if info_item in OLS_input:

            OLS_input[info_item] += data
        else:
            OLS_input[info_item] = data

    ### Get variables from balance sheet ###



    for info_item in balance_sheet_info_check:
        if(info_item in balance_sheet["Unnamed: 0"].values):
            
            data =  balance_sheet[balance_sheet["Unnamed: 0"] == info_item].values[0][1::]
            data = data.tolist()
            if len(data) != 4:
                print("Balance SHEET OOF", file)
            # data = transformer.transform(data.reshape(1, -1))
            # data = data.tolist()    
        else:
            data = [None] * (len(balance_sheet.columns)-1)

        if info_item in OLS_input:
            OLS_input[info_item] += data
        else:
            OLS_input[info_item] = data
            
    
    ### Get the returns of the stock over the quater
    
    times = balance_sheet.columns.values[1::]
    times = list(map(datetime.fromisoformat, times))
    stock_price_history = pd.read_csv(path + "/stock_prices.csv")
    stock_price_history["Date"] = pd.to_datetime(stock_price_history["Date"])
    quarterly_returns = []


    for i in range(len(times)):
        quarterly_data = pd.DataFrame()
        quarterly_data = stock_price_history[(stock_price_history["Date"] < times[i] ) & (stock_price_history["Date"] > times[i] - dateutil.relativedelta.relativedelta(months=3))].reset_index(drop=True)
        #calculate the net return in each quater
        if not(quarterly_data.empty) :
            total_ret = (quarterly_data.loc[len(quarterly_data)-1]["Close"] - quarterly_data.loc[0]["Open"])/quarterly_data.loc[0]["Open"]
            quarterly_returns.append(total_ret)
        else:
            quarterly_returns.append(None)
        
    if len(quarterly_returns) != 4:
        print("RETURN OOF", file)

    if "Returns" not in OLS_input:
        OLS_input["Returns"] = quarterly_returns
    else:
        OLS_input["Returns"] += quarterly_returns

  
for key , val in OLS_input.items():
    print(key , len(val))

df = pd.DataFrame(OLS_input)
df.to_csv(os.path.join(os.getcwd() , "Data") + "/Processed/linear_reg_data.csv")

Cash flow OOF 259960.KS
Cash flow OOF 259960.KS
Cash flow OOF 259960.KS
Cash flow OOF 259960.KS
Cash flow OOF 259960.KS
Cash flow OOF 259960.KS
Balance SHEET OOF 259960.KS
Balance SHEET OOF 259960.KS
Balance SHEET OOF 259960.KS
Balance SHEET OOF 259960.KS
Balance SHEET OOF 259960.KS
Balance SHEET OOF 259960.KS
RETURN OOF 259960.KS
